In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import argparse
import sys
from typing import List

import cv2
import pyttsx3
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\PartZ\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
def draw_boxes(image_bgr, results, score_threshold=0.5):
    """Draw bounding boxes and labels on a BGR image using YOLO results."""
    out = image_bgr.copy()
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            if conf < score_threshold:
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cls = r.names[cls_id]
            cv2.rectangle(out, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"{cls}: {conf:.2f}"
            ((text_w, text_h), _) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            cv2.rectangle(out, (x1, y1 - text_h - 6), (x1 + text_w, y1), (0, 255, 0), -1)
            cv2.putText(out, text, (x1, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
    return out

def run_image_mode(input_path: str, score: float = 0.5, output_path: str = None, weights: str = "yolov8n.pt"):
    model = YOLO(weights)
    img = cv2.imread(input_path)
    if img is None:
        print(f"Error: could not read {input_path}")
        return
    results = model(img)
    drawn = draw_boxes(img, results, score)
    if output_path:
        cv2.imwrite(output_path, drawn)
        print(f"Wrote annotated image to {output_path}")
    else:
        cv2.imshow('detected', drawn)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def main():
    args = parse_args()
    print('Loading YOLOv8 model...')
    model = YOLO(args.weights)
    print('Model loaded.')
    if args.mode == 'image':
        if not args.input:
            print('There should be an image in input !')
            sys.exit(1)
        run_image_mode(args.input, model, args.score, args.output)


In [6]:
run_image_mode("./images/home-1.jfif")


0: 448x640 1 chair, 1 couch, 1 bed, 150.1ms
Speed: 10.5ms preprocess, 150.1ms inference, 17.8ms postprocess per image at shape (1, 3, 448, 640)
